In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from constants import download_dir, output_dir
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import auc

In [2]:
# import mlflow.sklearn

In [3]:
# mlflow.set_tracking_uri("http://localhost:5000")

In [28]:
import os
os.getcwd()
print(output_dir)
path_monthly_1m = os.path.join(output_dir,"BTCUSDT-1m-monthly_data","BTCUSDT-1m.csv")
path_daily_1m = os.path.join(output_dir,"BTCUSDT-1m-daily_data","BTCUSDT-1m.csv")
print(path_monthly_1m)

D:\KISHORE\Binance-Data-Downloader\extracted_data
D:\KISHORE\Binance-Data-Downloader\extracted_data\BTCUSDT-1m-monthly_data\BTCUSDT-1m.csv


In [29]:
df = pd.read_csv(path_monthly_1m)

In [30]:
df.drop(["open_time", "close_time"], axis=1, inplace=True)

In [31]:
df.head()

,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,entry,...,PLUS_DM_233,ROC_233,ROCP_233,ROCR_233,ROCR100_233,RSI_233,ATR_233,NATR_233,BETA_233,CORREL_233
0,7182.43,7182.44,7178.75,7179.01,70.909,509145.78482,140,32.597,234063.27884,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7179.01,7179.01,7175.25,7177.93,99.420,713539.55348,148,16.311,117066.92118,7179.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7177.77,7182.60,7177.00,7181.11,69.330,497793.35929,104,43.723,313920.02981,7177.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7179.10,7179.10,7172.94,7175.25,97.368,698627.39382,193,36.616,262734.68999,7181.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7174.71,7177.14,7173.28,7175.61,33.725,241979.50880,124,15.885,113984.45772,7175.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
column_names = df.columns


missing_df = pd.DataFrame({
    "name" : column_names,
    "missing": df.isnull().mean().tolist()
}, columns=["name", "missing"])

missing_df["missing"] = missing_df["missing"] * 100

missing_df.sort_values("missing", ascending=False, inplace=True)


In [33]:
missing_df[:10]

,name,missing
19,entered_before,99.888391
17,longs_win_after,55.971509
16,shorts_win_after,55.041241
559,ADXR_233,0.044326
555,TEMA_233,0.044262
523,ADXR_200,0.038030
519,TEMA_200,0.037966
558,ADX_233,0.029572
548,DEMA_233,0.029508
487,ADXR_144,0.027346


In [34]:
df.drop(["entered_before", "longs_win_after", "shorts_win_after"], axis=1, inplace=True)

In [35]:
df.replace(np.nan, -500, inplace=True)

In [36]:
df.if_long.value_counts()

-1    880043
 1    692326
 0        81
Name: if_long, dtype: int64

In [37]:
df["if_long"] = df["if_long"].replace(-1, 0)
df["if_short"] = df["if_short"].replace(-1, 0)

In [38]:
df.if_long.value_counts()

0    880124
1    692326
Name: if_long, dtype: int64

In [39]:
df.if_short.value_counts()

0    865496
1    706954
Name: if_short, dtype: int64

In [40]:
int_cols = df.select_dtypes(include='int64').columns
df[int_cols] = df[int_cols].astype(float)


In [41]:
dtype_dict = {}
for col in df.columns:
    col_dtype = df[col].dtype.name
    if col_dtype not in dtype_dict:
        dtype_dict[col_dtype] = [col]
    else:
        dtype_dict[col_dtype].append(col)
print(dtype_dict)

{'float64': ['open', 'high', 'low', 'close', 'volume', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'entry', 'if_short', 'if_long', 'long_target', 'short_target', 'long_stop_loss', 'short_stop_loss', 'dual_loss', 'HT_TRENDLINE', 'SAR', 'SAREXT', 'T3', 'APO', 'BOP', 'MACD', 'MACD_signal', 'MACD_hist', 'PPO', 'TRIX', 'ULTOSC', 'WILLR', 'AD', 'ADOSC', 'OBV', 'HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR_inphase', 'HT_PHASOR_quadrature', 'HT_TRENDMODE', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE', 'TRANGE', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDL

## Long Model

In [42]:
# mlflow.sklearn.autolog()

In [43]:
long_df = df.drop("if_short", axis=1)

In [44]:
long_y = df["if_long"]

In [45]:
long_x = long_df.drop("if_long", axis=1)

In [46]:
columns_to_drop = [
    'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'count',
    'taker_buy_volume', 'taker_buy_quote_volume'
]

In [47]:
long_x.drop(columns_to_drop, axis=1, inplace=True)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(long_x, long_y, test_size=0.1)

In [49]:
long_clf_1m_more_indicators = RandomForestClassifier(max_depth=100, random_state=0, oob_score=True)

In [50]:
long_clf_1m_more_indicators.fit(X_train, y_train)

MemoryError: Unable to allocate 5.40 MiB for an array with shape (1415205,) and data type int32

In [ ]:
import pickle

# Save the model to a file
with open("long_clf_1m_more_indicators.pkl", "wb") as f:
    pickle.dump(long_clf_1m_more_indicators, f)


In [ ]:
short_df = df.drop("if_long", axis=1)

In [ ]:
short_y = df["if_short"]

In [ ]:
short_x = short_df.drop("if_short", axis=1)

In [ ]:
short_x.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(short_x, short_y, test_size=0.1)

In [ ]:
short_clf = RandomForestClassifier(max_depth=100, random_state=0, oob_score=True)

In [ ]:
short_clf.fit(X_train, y_train)